In [ ]:
import requests
from io import StringIO
import pandas as pd
from time import sleep

def read_df(string, sep="\t"):
    return pd.read_csv(StringIO(string), sep=sep)

root_url = "https://www.funselektorfun.com/artofrally/leaderboard/"

countries = ["Finland", "Sardinia", "Germany", "Kenya", "Norway", "Japan", "Indonesia"]
conditions = ["Dry", "Wet"]
stage_strs = [f"Stage_{n+1}_Forward" for n in range(6)] + [f"Stage_{n+1}_Reverse" for n in range(6)]

group = "Bonus_Logging"

In [ ]:
cars = []
times = []
names = []

for c in countries: 
    print('loading ' + c)

    for s in stage_strs: 
        for w in conditions: 

            stage_str = f"{c}_{s}_{w}_{group}"
            request_url = root_url + stage_str + "/0/6"

            response = requests.get(request_url)
            
            if response.text[0] == "<":
                response = requests.get(request_url)

            num_entries = len(eval(response.text)['leaderboard'])

            cars.append([eval(response.text)['leaderboard'][x]['carID'] for x in range(num_entries)] + [group, c])
            names.append([eval(response.text)['leaderboard'][x]['userName'] for x in range(num_entries)] + [group, c])
            
            for x in range(num_entries):
                resp = eval(response.text)['leaderboard'][x]
                times.append([group, c, resp['carID'], resp['score'], resp['rank'], resp['userName'], s, f"{c}_{s}_{w}", w])

        sleep(5)

    sleep(5)

time_df = pd.DataFrame(times)
time_df.columns = ['group', 'country', 'car', 'time', 'pos', 'driver', 'stage', 'full_stage', 'weather']

time_df.head(5)

Finland
Sardinia
Germany
Kenya
Norway
Japan
Indonesia


,group,country,car,time,pos,driver,stage,full_stage,weather
0,Bonus_Logging,Finland,0,92696,1,Proporo,Stage_1_Forward,Finland_Stage_1_Forward_Dry,Dry
1,Bonus_Logging,Finland,0,92933,2,Wesley Daggleston,Stage_1_Forward,Finland_Stage_1_Forward_Dry,Dry
2,Bonus_Logging,Finland,0,93359,3,Myth,Stage_1_Forward,Finland_Stage_1_Forward_Dry,Dry
3,Bonus_Logging,Finland,0,98131,4,Drumhater,Stage_1_Forward,Finland_Stage_1_Forward_Dry,Dry
4,Bonus_Logging,Finland,0,99167,5,Colorcat,Stage_1_Forward,Finland_Stage_1_Forward_Dry,Dry


In [ ]:
agg_df = time_df.groupby(['driver', 'pos']).count()[['group']].reset_index().sort_values(by=['pos', 'group'], ascending=[True, False])#.head(20)
agg_df.columns = ['driver', 'pos', 'count']

agg_df.head(10)

,driver,pos,count
366,Wesley Daggleston,1,167
265,Proporo,1,1
266,Proporo,2,25
335,Tezzaioli,2,23
112,Franky M.,2,16
103,Foxtheboss,2,13
322,Takaty-,2,11
325,TemplarLad,2,9
457,lFINlcoffeejack,2,8
232,Myth,2,7


In [9]:
time_df.query('driver != "Wesley Daggleston" & pos == 1')

,group,country,car,time,pos,driver,stage,full_stage,weather
0,Bonus_Logging,Finland,0,92696,1,Proporo,Stage_1_Forward,Finland_Stage_1_Forward_Dry,Dry
